In [229]:
import face_recognition
from PIL import Image, ImageDraw
import os, sys, io
import glob
from google.cloud import vision
import numpy as np
import pandas as pd

In [303]:
img_dir = "./img/known/" 
data_path = os.path.join(img_dir,'*.jpg')
files = glob.glob(data_path)
data = []

In [304]:
for f1 in files:
    img = face_recognition.load_image_file(f1)
    data.append(face_recognition.face_encodings(img)[0])

In [324]:
unknown_image_location = './img/unknown/mohanlal-kireedam.jpg'

In [325]:
test_image = face_recognition.load_image_file(unknown_image_location)

In [326]:
face_location = face_recognition.face_locations(test_image)
face_encodings = face_recognition.face_encodings(test_image,face_location)

In [327]:
pil_image = Image.fromarray(test_image)

In [328]:
draw = ImageDraw.Draw(pil_image)

In [329]:
matches = []
name_of_person = []
for (top,bottom,left,right) , face_encoding in zip(face_location,face_encodings):
    matches = face_recognition.compare_faces(data,face_encoding)
    name = "unknown"
    print(matches)
    name_of_person.append(check_fun(matches))

[True, False, False, True, False, False, False, False, False]


In [311]:
def check_fun(matches):
    if True in matches:
        first_match_index = matches.index(True)
        name = files[first_match_index].split('\\')[1].split('.')[0]
        draw.rectangle(((left,top),(right,bottom)),outline=(0,0,0))

        text_width,text_height = draw.textsize(name)

        draw.rectangle(((left ,bottom ),(right, bottom)),fill= (0,0,0),outline=(0,0,0))

        draw.text((left ,bottom - text_height ),name, fill=(255,255,255))

        return name

    else:
        name = str(input("Enter the name"))
        pil_image.save("./img/known/{0}".format(name)) 

In [330]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'ServiceAccountToken.json'

In [331]:
client = vision.ImageAnnotatorClient()

In [332]:
with io.open(unknown_image_location,'rb') as image_file:
    content = image_file.read()

In [333]:
image_to_find_face_details = vision.types.Image(content = content)

In [334]:
response = client.face_detection(image = image_to_find_face_details)

In [335]:
face_annotation = response.face_annotations

In [336]:
face_emotion = ['joy','sorrow','anger','surprise','underexposed','blurred','headwear']

In [337]:
emmotion_of_person = list()
for face in face_annotation:
        b = [face.joy_likelihood,
                face.sorrow_likelihood,
                face.anger_likelihood,
                face.surprise_likelihood,
                face.under_exposed_likelihood,
                face.blurred_likelihood,
                face.headwear_likelihood]
        emmotion_of_person.append(face_emotion[b.index(max(b))])

In [338]:
emmotion_of_person

['joy']

In [339]:
df = pd.DataFrame(columns = ['Name','Emmotion'])

In [340]:
for name,emmotion in zip(name_of_person,emmotion_of_person):
    df = df.append(dict(
        Name = name,
        Emmotion = emmotion
    ),ignore_index=True)

In [341]:
df

,Name,Emmotion
0,mammootty,joy
